In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Sales.csv")
# Display original data
df

,Order ID,Customer Name,Order Date,Product,Quantity,Unit Price,Total Revenue
0,1001,John Doe,01/01/2024,Widget A,10.0,25.0,250.0
1,1002,Jane Smith,01/02/2024,Widget B,5.0,40.0,200.0
2,1003,NaN,2024/01/03',Widget A,NaN,25.0,NaN
3,1004,Alice Johnson,04/01/2024,Widget C,3.0,NaN,210.0
4,1005,Bob Brown,2024/01/05',Widget B,10.0,40.0,400.0
5,1006,John Doe,06/01/2024,Widget A,4.0,25.0,100.0
6,1001,John Doe,01/01/2024,Widget A,10.0,25.0,250.0
7,1007,Jane Smith,07/01/2024,Widget C,-6.0,70.0,-420.0


In [3]:
# 1. Handle missing values

In [4]:
# Fill missing customer names with 'Unknown'
df['Customer Name'].fillna('Unknown', inplace=True)
df

/tmp/ipykernel_6998/1713562261.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Customer Name'].fillna('Unknown', inplace=True)


,Order ID,Customer Name,Order Date,Product,Quantity,Unit Price,Total Revenue
0,1001,John Doe,01/01/2024,Widget A,10.0,25.0,250.0
1,1002,Jane Smith,01/02/2024,Widget B,5.0,40.0,200.0
2,1003,Unknown,2024/01/03',Widget A,NaN,25.0,NaN
3,1004,Alice Johnson,04/01/2024,Widget C,3.0,NaN,210.0
4,1005,Bob Brown,2024/01/05',Widget B,10.0,40.0,400.0
5,1006,John Doe,06/01/2024,Widget A,4.0,25.0,100.0
6,1001,John Doe,01/01/2024,Widget A,10.0,25.0,250.0
7,1007,Jane Smith,07/01/2024,Widget C,-6.0,70.0,-420.0


In [5]:
# For missing unit price, we can calculate it from total revenue and quantity if possible
mask = (df['Unit Price'].isna()) & (df['Total Revenue'].notna()) & (df['Quantity'].notna())
df.loc[mask, 'Unit Price'] = df.loc[mask, 'Total Revenue'] / df.loc[mask, 'Quantity']
df

,Order ID,Customer Name,Order Date,Product,Quantity,Unit Price,Total Revenue
0,1001,John Doe,01/01/2024,Widget A,10.0,25.0,250.0
1,1002,Jane Smith,01/02/2024,Widget B,5.0,40.0,200.0
2,1003,Unknown,2024/01/03',Widget A,NaN,25.0,NaN
3,1004,Alice Johnson,04/01/2024,Widget C,3.0,70.0,210.0
4,1005,Bob Brown,2024/01/05',Widget B,10.0,40.0,400.0
5,1006,John Doe,06/01/2024,Widget A,4.0,25.0,100.0
6,1001,John Doe,01/01/2024,Widget A,10.0,25.0,250.0
7,1007,Jane Smith,07/01/2024,Widget C,-6.0,70.0,-420.0


In [6]:
# For missing total revenue, we can calculate it from quantity and unit price if possible
mask = (df['Total Revenue'].isna()) & (df['Unit Price'].notna()) & (df['Quantity'].notna())
df.loc[mask, 'Total Revenue'] = df.loc[mask, 'Quantity'] * df.loc[mask, 'Unit Price']
df

,Order ID,Customer Name,Order Date,Product,Quantity,Unit Price,Total Revenue
0,1001,John Doe,01/01/2024,Widget A,10.0,25.0,250.0
1,1002,Jane Smith,01/02/2024,Widget B,5.0,40.0,200.0
2,1003,Unknown,2024/01/03',Widget A,NaN,25.0,NaN
3,1004,Alice Johnson,04/01/2024,Widget C,3.0,70.0,210.0
4,1005,Bob Brown,2024/01/05',Widget B,10.0,40.0,400.0
5,1006,John Doe,06/01/2024,Widget A,4.0,25.0,100.0
6,1001,John Doe,01/01/2024,Widget A,10.0,25.0,250.0
7,1007,Jane Smith,07/01/2024,Widget C,-6.0,70.0,-420.0


In [7]:
# 2. Fix inconsistent date formats

In [8]:
# Lets clean the dates by removing any trailing quotes and standardizing the format
df['Order Date'] = df['Order Date'].str.replace("'","")
# We then parse dates with explicit format handling
def parse_date(date_str):
    try:
        #We try MM/DD/YYYY format first (for most dates)
        return pd.to_datetime(date_str, format='%m/%d/%Y')
    except ValueError:
        try:
            #We  Try YYYY/MM/DD format for the others
            return pd.to_datetime(date_str, format='%Y/%m/%d')
        except ValueError:
            return pd.NaT

df['Order Date'] = df['Order Date'].apply(parse_date)
df

,Order ID,Customer Name,Order Date,Product,Quantity,Unit Price,Total Revenue
0,1001,John Doe,2024-01-01,Widget A,10.0,25.0,250.0
1,1002,Jane Smith,2024-01-02,Widget B,5.0,40.0,200.0
2,1003,Unknown,2024-01-03,Widget A,NaN,25.0,NaN
3,1004,Alice Johnson,2024-04-01,Widget C,3.0,70.0,210.0
4,1005,Bob Brown,2024-01-05,Widget B,10.0,40.0,400.0
5,1006,John Doe,2024-06-01,Widget A,4.0,25.0,100.0
6,1001,John Doe,2024-01-01,Widget A,10.0,25.0,250.0
7,1007,Jane Smith,2024-07-01,Widget C,-6.0,70.0,-420.0


In [9]:
# Lets drop rows where date couldn't be parsed (though in this case none of the rows have dates that couldn't be parsed)
df.dropna(subset=['Order Date'], inplace=True)
df

,Order ID,Customer Name,Order Date,Product,Quantity,Unit Price,Total Revenue
0,1001,John Doe,2024-01-01,Widget A,10.0,25.0,250.0
1,1002,Jane Smith,2024-01-02,Widget B,5.0,40.0,200.0
2,1003,Unknown,2024-01-03,Widget A,NaN,25.0,NaN
3,1004,Alice Johnson,2024-04-01,Widget C,3.0,70.0,210.0
4,1005,Bob Brown,2024-01-05,Widget B,10.0,40.0,400.0
5,1006,John Doe,2024-06-01,Widget A,4.0,25.0,100.0
6,1001,John Doe,2024-01-01,Widget A,10.0,25.0,250.0
7,1007,Jane Smith,2024-07-01,Widget C,-6.0,70.0,-420.0


In [10]:
# 3. Remove duplicate rows
df.drop_duplicates(inplace=True)
df

,Order ID,Customer Name,Order Date,Product,Quantity,Unit Price,Total Revenue
0,1001,John Doe,2024-01-01,Widget A,10.0,25.0,250.0
1,1002,Jane Smith,2024-01-02,Widget B,5.0,40.0,200.0
2,1003,Unknown,2024-01-03,Widget A,NaN,25.0,NaN
3,1004,Alice Johnson,2024-04-01,Widget C,3.0,70.0,210.0
4,1005,Bob Brown,2024-01-05,Widget B,10.0,40.0,400.0
5,1006,John Doe,2024-06-01,Widget A,4.0,25.0,100.0
7,1007,Jane Smith,2024-07-01,Widget C,-6.0,70.0,-420.0


In [11]:
# 4. Fix wrong data

In [12]:
# Negative quantity doesn't make sense - we'll take absolute value and note this in the data
df['Quantity'] = df['Quantity'].abs()

In [13]:
# Negative revenue would only make sense for returns, but without a returns column, we'll make positive
df['Total Revenue'] = df['Total Revenue'].abs()
df

,Order ID,Customer Name,Order Date,Product,Quantity,Unit Price,Total Revenue
0,1001,John Doe,2024-01-01,Widget A,10.0,25.0,250.0
1,1002,Jane Smith,2024-01-02,Widget B,5.0,40.0,200.0
2,1003,Unknown,2024-01-03,Widget A,NaN,25.0,NaN
3,1004,Alice Johnson,2024-04-01,Widget C,3.0,70.0,210.0
4,1005,Bob Brown,2024-01-05,Widget B,10.0,40.0,400.0
5,1006,John Doe,2024-06-01,Widget A,4.0,25.0,100.0
7,1007,Jane Smith,2024-07-01,Widget C,6.0,70.0,420.0


In [14]:
# 5. Check for unnecessary columns
#All columns seem relevant to me

In [15]:
# Finally, data type conversions
df['Order ID'] = df['Order ID'].astype(int)
df['Quantity'] = df['Quantity'].fillna(1).astype(int)
df['Total Revenue'] = df['Total Revenue'].fillna(df['Total Revenue'].mean())
df

,Order ID,Customer Name,Order Date,Product,Quantity,Unit Price,Total Revenue
0,1001,John Doe,2024-01-01,Widget A,10,25.0,250.000000
1,1002,Jane Smith,2024-01-02,Widget B,5,40.0,200.000000
2,1003,Unknown,2024-01-03,Widget A,1,25.0,263.333333
3,1004,Alice Johnson,2024-04-01,Widget C,3,70.0,210.000000
4,1005,Bob Brown,2024-01-05,Widget B,10,40.0,400.000000
5,1006,John Doe,2024-06-01,Widget A,4,25.0,100.000000
7,1007,Jane Smith,2024-07-01,Widget C,6,70.0,420.000000


In [16]:
# Resetting index after all cleaning
df.reset_index(drop=True, inplace=True)

In [17]:
# Display cleaned data
df

,Order ID,Customer Name,Order Date,Product,Quantity,Unit Price,Total Revenue
0,1001,John Doe,2024-01-01,Widget A,10,25.0,250.000000
1,1002,Jane Smith,2024-01-02,Widget B,5,40.0,200.000000
2,1003,Unknown,2024-01-03,Widget A,1,25.0,263.333333
3,1004,Alice Johnson,2024-04-01,Widget C,3,70.0,210.000000
4,1005,Bob Brown,2024-01-05,Widget B,10,40.0,400.000000
5,1006,John Doe,2024-06-01,Widget A,4,25.0,100.000000
6,1007,Jane Smith,2024-07-01,Widget C,6,70.0,420.000000


In [18]:
# Data summary
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order ID       7 non-null      int64         
 1   Customer Name  7 non-null      object        
 2   Order Date     7 non-null      datetime64[ns]
 3   Product        7 non-null      object        
 4   Quantity       7 non-null      int64         
 5   Unit Price     7 non-null      float64       
 6   Total Revenue  7 non-null      float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 520.0+ bytes
None


In [19]:
print(df.describe())

          Order ID                     Order Date   Quantity  Unit Price  \
count     7.000000                              7   7.000000    7.000000   
mean   1004.000000  2024-03-02 17:08:34.285714176   5.571429   42.142857   
min    1001.000000            2024-01-01 00:00:00   1.000000   25.000000   
25%    1002.500000            2024-01-02 12:00:00   3.500000   25.000000   
50%    1004.000000            2024-01-05 00:00:00   5.000000   40.000000   
75%    1005.500000            2024-05-01 12:00:00   8.000000   55.000000   
max    1007.000000            2024-07-01 00:00:00  10.000000   70.000000   
std       2.160247                            NaN   3.408672   20.177781   

       Total Revenue  
count       7.000000  
mean      263.333333  
min       100.000000  
25%       205.000000  
50%       250.000000  
75%       331.666667  
max       420.000000  
std       113.235252  


In [20]:
# Save cleaned data to new CSV
df.to_csv('cleaned_sales_data.csv', index=False)